## CRF What's Wrong with Participants? --II

In [1]:
from crf import *
from crf_support import get_all_data
import sys,os,pickle

import os, time, pprint

from features_generator import abstracts2features, get_genia_tags, sanity_check

from gensim.models import Word2Vec

/Users/Jing/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Jing/anaconda/envs/tensorflow/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
tag = 'P'
eval_tags = [tag]

Get data

In [3]:
# Get train data
train_tokens, train_tags = get_all_data('train', tag)
train_genia_tags = get_genia_tags('train')

In [4]:
# Get dev data
dev_tokens, dev_tags = get_all_data('dev', tag)
dev_genia_tags = get_genia_tags('dev')

In [5]:
# Get test data
test_tokens, test_tags = get_all_data('test', tag)
test_genia_tags = get_genia_tags('test')

In [6]:
# compare bag of words in train and dev, and test

def bagofwords(dat):
    dat_set=set()
    for abstract in dat:
        dat_set.update(abstract)
    return dat_set


train_set = bagofwords(train_tokens)
dev_set = bagofwords(dev_tokens)
test_set = bagofwords(test_tokens)


In [18]:
feature_name='OneNegOneHot'
file_name='crf_results/{}'.format(feature_name)
f = open(file_name + '_results.txt','r')
result=pickle.load(f)
f.close()
result

{(0.0001, 0.0001): [0.52634933593511524,
  0.43697861157889167,
  0.47715768242587625],
 (0.0001, 0.001): [0.55854056838367594,
  0.44449206574046446,
  0.49487034840599514],
 (0.0001, 0.01): [0.60911415766341093,
  0.45603869820708953,
  0.52143931607158778],
 (0.0001, 0.1): [0.68131797232957758,
  0.46206041852235979,
  0.55063603276601092],
 (0.0001, 1): [0.75078030989922828, 0.46865655460635064, 0.57696627029307224],
 (0.001, 0.0001): [0.54642052201828384,
  0.43682664642721314,
  0.48550355155305897],
 (0.001, 0.001): [0.55921956396949013,
  0.44838277706991364,
  0.49753169823310722],
 (0.001, 0.01): [0.61454398670159172,
  0.45335926829725703,
  0.52171860996215913],
 (0.001, 0.1): [0.68264718102813748, 0.4635564626920633, 0.55213785235848623],
 (0.001, 1): [0.75690910229299357, 0.46447865196054972, 0.57558528966335065],
 (0.01, 0.0001): [0.59386039484298236,
  0.45955267946253187,
  0.51803234679204246],
 (0.01, 0.001): [0.60655275609492665,
  0.45538841751451431,
  0.519925419

In [8]:
#count the words that are not in train dataset:

dev_miss = 0
for word in dev_set:
    if word not in train_set:
        dev_miss+=1

test_miss=0
for word in test_set:
    if word not in train_set:
        test_miss+=1

        
print "There are %s words in Dev data, and %s unseen words (%s)" %(len(dev_set),dev_miss,np.float(dev_miss)/len(dev_set)*100)
print "There are %s words in Test data, and %s unseen words (%s)" %(len(test_set),test_miss,np.float(test_miss)/len(test_set)*100)
        

There are 22646 words in Dev data, and 7818 unseen words (34.5226530072)
There are 14655 words in Test data, and 3895 unseen words (26.5779597407)


In [35]:
# Predict dev tags
pred_dev_tags = predict_tags(tagger, dev_features)

#evaluate result of the words that have seen.

def seen_words_pred(tokens, tags, pred_tags):
    seen_tags =[]
    seen_pred_tags=[]
    for i,abstract in enumerate(tokens):
        dev=[]
        pred=[]
        for j, word in enumerate(abstract):
            if word in train_set:
                dev.append(tags[i][j])
                pred.append(pred_tags[i][j])
        seen_tags.append(dev)
        seen_pred_tags.append(pred)
    return seen_tags, seen_pred_tags

seen_dev_tags, seen_pred_dev_tags = seen_words_pred(dev_tokens,dev_tags,pred_dev_tags)
# Evaluate dev tags
dev_result = evaluate_prediction(seen_pred_dev_tags, seen_dev_tags, eval_tags)
print 'dev:'
print_result(dev_result)
sys.stdout.flush()

# Predict test tags
pred_test_tags = predict_tags(tagger, test_features)

seen_test_tags, seen_pred_test_tags = seen_words_pred(test_tokens,test_tags,pred_test_tags)

# # Evaluate test tags
test_result = evaluate_prediction(seen_pred_test_tags, seen_test_tags, eval_tags)
print 'test:'
print_result(test_result)
sys.stdout.flush()

dev:
P: (0.7424930423319174, 0.48120372128346306, 0.5839525372962386)
test:
P: (0.7572129255626082, 0.5026333429091258, 0.6042014388489209)


**If we limit to the words that have seen in the dataset, the F1 score is slightly better. Main problem is not unseen words.**

Compute features

In [77]:
# result of the one neg one hot, no other features

dev:
P: (0.746385029770343, 0.4764274726269116, 0.5816072908036454)
train:
P: (0.8331695473726805, 0.5432444656632409, 0.6576723882242891)
test:
P: (0.7552387740555951, 0.4816801527411583, 0.5882091706450538)


1. Tested features that include features of the neghbors, overfitting, got worse result on predictions (0.07).

2. Test Features with only the ones for the word itself. 

In [87]:
"""One Negh with the word's features
options_string = 'left_neighbors=1 right_neighbors=1 one_hot \
one_hot_neighbors inside_paren pos chunk iob named_entity isupper istitle'

w2v=None

features_name = 'Feat7OneNeg'
"""

dev:
P: (0.7288770053475936, 0.4933490181883992, 0.5884194053208137)
train:
P: (0.8203448339860062, 0.5553308038294343, 0.6623112942498595)
test:
P: (0.7284967320261438, 0.5066824256750614, 0.5976727974690332)


3. Test Features with only the ones for the word itself, with w2v.

In [103]:
"""
options_string = 'left_neighbors=1 right_neighbors=1 w2v_model=pubmed w2v \
w2v_neighbors w2v_size=10 inside_paren pos chunk iob named_entity isupper istitle'

w2v=True

features_name = 'Feat7OneNegw2v'
"""

dev:
P: (0.4004106776180698, 0.02646819292371731, 0.04965411874549081)
train:
P: (0.3377854373115037, 0.019699977385230043, 0.037228738306662235)
test:
P: (0.38258164852255055, 0.022365669606327847, 0.04226077993471912)


4. Test Features with word itself and neigbors, with one hot.

In [129]:
"""
options_string = 'left_neighbors=1 right_neighbors=1 one_hot \
one_hot_neighbors inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors '

w2v=True

features_name = 'FeatAllOneNegFeatAll'
"""


dev:
P: (0.7433616111455963, 0.48764817663559856, 0.5889456572224803)
train:
P: (0.866936073727326, 0.5956604769203708, 0.7061408081500126)
test:
P: (0.7543541869862059, 0.4922265660514592, 0.5957306338028169)


In [9]:
# Set options
big_options_string = 'left_neighbors=3 right_neighbors=3 inside_paren pos chunk iob named_entity \
inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors \
one_hot one_hot_neighbors w2v_model=pubmed w2v w2v_neighbors w2v_size=10 cosine_simil cosine_simil_neighbors \
isupper isupper_neighbors istitle istitle_neighbors'

options_string = 'left_neighbors=1 right_neighbors=1 one_hot \
one_hot_neighbors'

w2v=False

features_name = 'OneNeig'

In [10]:
# Compute features for train
train_features = abstracts2features(train_tokens, train_genia_tags, w2v=w2v, options_string=options_string)

3499: ['Pulsed', 'azithromycin', 'treatment']


In [11]:
# Compute features for dev
dev_features = abstracts2features(dev_tokens, dev_genia_tags, w2v=w2v, options_string=options_string)

999: ['Serum', 'bactericidal', 'activities']


In [12]:
# Compute features for test
test_features = abstracts2features(test_tokens, test_genia_tags, w2v=w2v, options_string=options_string)

500: ['Efficacy', 'and', 'cost-effectiveness']


In [13]:
# For debug
sanity_check(train_features)

Number of abstracts: 3500
Number of tokens:    927022
Number of features:  2774066 

Avg tokens per abstract: 264
Avg features per token:  2 

Max features per token:  3
Min features per token:  2


Grid search

In [14]:
# # Run grid search
# grid_file_name = 'crf_results/{}_grid'.format(features_name)
# num_iters = 100
# l1_list = [ 0.0001, 0.001, 0.01, 0.1, 1]
# l2_list = [ 0.0001, 0.001, 0.01, 0.1, 1]

# start_time = time.time()
# grid_search_result = grid_search(train_features, train_tags, dev_features, dev_tags,\
#                                  num_iters, l1_list, l2_list, eval_tags, file_name=grid_file_name, save=True)
# print("--- %s seconds ---" % (time.time() - start_time))

# os.remove(grid_file_name + '.model')

In [15]:
# Sort result
# sorted_result = sort_by_metric(grid_search_result, tag, metric='f1')
# print_result(sorted_result)

Run CRF

In [19]:
# Set options
num_iters = 100
l1 = 1
l2 = 0.1
file_name = 'crf_results/{}'.format(features_name)

In [21]:
# Run CRF
start_time = time.time()
crf_result = get_crf_results(train_features, train_tags, dev_features, dev_tags, num_iters, l1, l2, eval_tags,
                             file_name=file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

Adding data...
Training model...
Done!
--- 40.0797998905 seconds ---


In [22]:
# Print result
print_result(crf_result)

P: (0.741732171942497, 0.4738937652701113, 0.5783066007785108)


Get model

In [23]:
# Get model from file
tagger = get_tagger(file_name)

In [24]:
# For debug
print_model_info(tagger)

Top likely transitions:
None   -> None    1.917295
P      -> P       1.903707
None   -> P       -1.233491
P      -> None    -3.488813

Top positive:
6.386364 None   word[0]:BACKGROUND
5.351200 None   word[0]:DESIGN
4.951513 None   word[0]:PARTICIPANTS
3.983060 None   word[1]:Eleven
3.441205 None   word[0]:METHODS
3.387782 None   word[1]:Nine
3.344130 None   word[0]:.
3.314683 None   word[0]:As
3.270989 None   word[0]:Both
3.260569 P      word[0]:alcoholics
3.107455 None   word[0]:However
3.074564 None   word[1]:Fifty-four
3.015337 None   word[-1]:hypertension.
2.921317 None   word[0]:PATIENTS
2.884139 None   word[0]:PURPOSE
2.835580 None   word[0]:Of
2.809665 None   word[0]:Overall
2.806916 None   word[0]:SUBJECTS
2.727374 None   word[0]:Results
2.699177 None   word[0]:In

Top negative:
-2.030994 None   word[1]:PURPOSE
-2.035106 P      word[0]:treating
-2.041772 P      word[0]:These
-2.071515 None   word[1]:OBJECTIVE
-2.151928 None   word[-1]:PARTICIPANTS
-2.166695 P      word[-1]:volu

Predict tags

In [25]:
# Predict dev tags
pred_dev_tags = predict_tags(tagger, dev_features)

# Evaluate dev tags
dev_result = evaluate_prediction(pred_dev_tags, dev_tags, eval_tags)
print 'dev:'
print_result(dev_result)
sys.stdout.flush()

# Predict train tags
pred_train_tags = predict_tags(tagger, train_features)

# Evaluate train tags
train_result = evaluate_prediction(pred_train_tags, train_tags, eval_tags)
print 'train:'
print_result(train_result)
sys.stdout.flush()

# Predict test tags
pred_test_tags = predict_tags(tagger, test_features)

# Evaluate test tags
test_result = evaluate_prediction(pred_test_tags, test_tags, eval_tags)
print 'test:'
print_result(test_result)
sys.stdout.flush()

dev:
P: (0.741732171942497, 0.4738937652701113, 0.5783066007785108)
train:
P: (0.8218648286425426, 0.5393120084428474, 0.6512622891127564)
test:
P: (0.7573180931140229, 0.49395399581780164, 0.5979199911957299)


In [36]:
# Predict dev tags
pred_dev_tags = predict_tags(tagger, dev_features)

#evaluate result of the words that have seen.

def seen_words_pred(tokens, tags, pred_tags):
    seen_tags =[]
    seen_pred_tags=[]
    for i,abstract in enumerate(tokens):
        dev=[]
        pred=[]
        for j, word in enumerate(abstract):
            if word in train_set:
                dev.append(tags[i][j])
                pred.append(pred_tags[i][j])
        seen_tags.append(dev)
        seen_pred_tags.append(pred)
    return seen_tags, seen_pred_tags

seen_dev_tags, seen_pred_dev_tags = seen_words_pred(dev_tokens,dev_tags,pred_dev_tags)
# Evaluate dev tags
dev_result = evaluate_prediction(seen_pred_dev_tags, seen_dev_tags, eval_tags)
print 'dev:'
print_result(dev_result)
sys.stdout.flush()

# Predict test tags
pred_test_tags = predict_tags(tagger, test_features)

seen_test_tags, seen_pred_test_tags = seen_words_pred(test_tokens,test_tags,pred_test_tags)

# # Evaluate test tags
test_result = evaluate_prediction(seen_pred_test_tags, seen_test_tags, eval_tags)
print 'test:'
print_result(test_result)
sys.stdout.flush()

dev:
P: (0.7424930423319174, 0.48120372128346306, 0.5839525372962386)
test:
P: (0.7572129255626082, 0.5026333429091258, 0.6042014388489209)


K-fold evaluation

In [25]:
# Run K-fold
kfold_file_name = 'crf_results/{}_kfold'.format(features_name)

start_time = time.time()
kfold_result = get_kfold_results(train_features, train_tags, num_iters, l1, l2, eval_tags,\
                                 file_name=kfold_file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

os.remove(kfold_file_name + '.model')

On fold 0
Adding data...
Training model...
Done!
On fold 1
Adding data...
Training model...
Done!
On fold 2
Adding data...
Training model...
Done!
On fold 3
Adding data...
Training model...
Done!
On fold 4
Adding data...
Training model...
Done!
--- 162.671914816 seconds ---


In [26]:
# Print all results
print_result(kfold_result)

Fold 0
P: (0.743448275862069, 0.4230492510955589, 0.5392471549460169)
Fold 1
P: (0.7338930105427567, 0.4609724691887915, 0.5662636990170602)
Fold 2
P: (0.712115031613298, 0.4341311781162574, 0.5394152408172724)
Fold 3
P: (0.7171599922615592, 0.4529569892473118, 0.5552310342245188)
Fold 4
P: (0.7343669781291714, 0.46014282956958114, 0.565778023890515)
Average
P: (0.72819665768177089, 0.44625054344350013, 0.5531870305790767)


In [41]:
def print_with_spaces(l, spaces):
    # This pads strings to be of space length and aligned left
    formatter = lambda space: '{:' + str(space) + '}'
    
    for sublist in l:
        print ''.join([formatter(space).format(string) for string, space in zip(sublist, spaces)])

In [55]:
i = 2
print_with_spaces(zip(dev_tokens[i], dev_tags[i], pred_dev_tags[i]), [25, 5, 5])

Netilmicin               None None 
in                       P    None 
the                      P    None 
neonate                  P    None 
:                        None None 
pharmacokinetic          None None 
analysis                 None None 
and                      None None 
influence                None None 
of                       None None 
parenteral               None None 
nutrition                None None 
.                        None None 
OBJECTIVE                None None 
The                      None None 
aim                      None None 
of                       None None 
this                     None None 
study                    None None 
was                      None None 
to                       None None 
investigate              None None 
the                      None None 
impact                   None None 
of                       None None 
parenteral               None None 
nutrition                None None 
on                       Non